In [1]:
try:
  # শুধুমাত্র টেন্সর-ফ্লো ২.x ব্যবহার করবো 
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
keras = tf.keras

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import gensim, re
import numpy as np
import pandas as pd

# একটা ডেটা ডিকশনারি বানাই, আমাদের পছন্দ মতো বাক্য 

data = ['আমি মেশিন লার্নিং শিখতে পছন্দ করি',
        'আমার বই পড়তে ভালো লাগে না',
        'পাইথন শিখতে কষ্ট',
        'এই বইটা বেশ ভালো লাগছে',
        'আমার ন্যাচারাল ল্যাঙ্গুয়েজ প্রসেসিং পছন্দ']

labels = ['positive', 'negative', 'negative', 'positive', 'positive']

# আমাদের ডেটাকে কিছুটা প্রি-প্রসেস করি, বাংলা ইউনিকোড রেঞ্জের বাইরে সবকিছু ফেলে দিচ্ছি 

text = [re.sub(r'[^\u0980-\u09FF ]+', '', sentence) for sentence in data]

In [4]:
text

['আমি মেশিন লার্নিং শিখতে পছন্দ করি',
 'আমার বই পড়তে ভালো লাগে না',
 'পাইথন শিখতে কষ্ট',
 'এই বইটা বেশ ভালো লাগছে',
 'আমার ন্যাচারাল ল্যাঙ্গুয়েজ প্রসেসিং পছন্দ']

In [5]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# আমাদের টোকেনাইজার কতগুলো ফিচার এক্সট্র্যাক্ট করবে?
features = 500
tokenizer = Tokenizer(num_words = features)

# আমাদের টোকেনাইজারকে পুরো টেক্সটে ফিট করতে হবে 
tokenizer.fit_on_texts(text)

# আমাদের টোকেনাইজার চেনে সেরকম শব্দ নিয়ে আসতে হবে 
word_index = tokenizer.word_index

# ম্যাট্রিক্স এর মধ্যে টোকেনকে ফেলি 
X = tokenizer.texts_to_sequences(text)
X = pad_sequences(X)

# লেবেল তৈরি করি 
y = np.asarray(pd.get_dummies(labels))

In [6]:
pd.get_dummies(labels)

,negative,positive
0,0,1
1,1,0
2,1,0
3,0,1
4,0,1


In [7]:
#train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

##  ফাস্টটেক্সট

In [17]:
from urllib.request import urlopen
import gzip

# ফাস্টটেক্সটের প্রি-ট্রেইন ভেক্টরগুলোকে নিয়ে আসি, বাংলায় - cc.bn.300.vec.gz ৯০০ মেগাবাইটের মতো 
file = gzip.open(urlopen('https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bn.300.vec.gz'))

In [18]:
vocab_and_vectors = {}
# শব্দগুলোকে ডিকশনারি ইনডেক্সে রাখি, আর ভেক্টরগুলোকে ওয়ার্ড ভ্যালুতে, একটু সময় নেবে  
for line in file:
  values = line.split()
  word = values[0].decode('utf-8')
  vector = np.asarray(values[1:], dtype='float32')
  # print(word)
  vocab_and_vectors[word] = vector

EOFError: Compressed file ended before the end-of-stream marker was reached

In [ ]:
# দেখি কি আছে এখানে?

list(vocab_and_vectors.keys())[2]

In [ ]:
# এমবেডিং ম্যাট্রিক্স 

embedding_matrix = np.zeros((len(vocab_and_vectors) + 1, 300))
for i, word, in enumerate(vocab_and_vectors.keys()):
  # print(i)
  # print(word)
  embedding_vector = vocab_and_vectors.get(word)
  # যে শব্দগুলো পাওয়া যাবে না, সেগুলোকে '০'তে রাখবো 
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix

In [ ]:
vocab_and_vectors

##  মডেল  

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

# লেয়ারকে ইনিশিয়ালাইজ করি, অবশ্যই সিকোয়েন্সিয়াল মডেল  
model = Sequential()
# শব্দ ভেক্টরকে এমবেড করি 
model.add(Embedding(len(vocab_and_vectors)+1, 300, input_length=X.shape[1], weights=[embedding_matrix], trainable=False))
# ডেটার মধ্যে কো-রিলেশন বোঝার চেষ্টা করতে দেই মডেলকে 
model.add(LSTM(300, return_sequences=False))
model.add(Dense(y.shape[1], activation="softmax"))
# মডেলের কাঠামো দেখি 
model.summary()

NameError: name 'embedding_matrix' is not defined

In [20]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['acc'])

In [21]:
batch = 32
epochs = 12
model.fit(X_train,y_train,batch,epochs)

Epoch 1/12


TypeError: in user code:

    File "C:\Users\user\anaconda3\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\user\anaconda3\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\user\anaconda3\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\user\anaconda3\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\user\anaconda3\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "C:\Users\user\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:\Users\

In [22]:
model.evaluate(X_test,y_test)

TypeError: in user code:

    File "C:\Users\user\anaconda3\lib\site-packages\keras\engine\training.py", line 1557, in test_function  *
        return step_function(self, iterator)
    File "C:\Users\user\anaconda3\lib\site-packages\keras\engine\training.py", line 1546, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\user\anaconda3\lib\site-packages\keras\engine\training.py", line 1535, in run_step  **
        outputs = model.test_step(data)
    File "C:\Users\user\anaconda3\lib\site-packages\keras\engine\training.py", line 1501, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\user\anaconda3\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "C:\Users\user\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:\Users\user\anac

##  প্রেডিকশন

In [23]:
# ইচ্ছেমতো বাক্য লিখে টেস্ট 

sents = ['আমি মেশিন লার্নিং শিখতে পছন্দ করি', 'পছন্দ করি', 'করি না', 'আমি ভাল', 'বেশ কষ্ট', 'এই বইটা বেশ ভালো লাগছে']
# sent_n = [[word_index[w]+3 for w in s.split()] for s in sents]
sent_n = tokenizer.texts_to_sequences(sents)
X_new = pad_sequences(sent_n)
prediction = model.predict(X_new)

print(prediction)

1/1 [==============================] - 0s 44ms/step
[[ 5  6  7  1  2  8]
 [ 0  0  0  0  2  8]
 [ 0  0  0  0  8 12]
 [ 0  0  0  0  0  5]
 [ 0  0  0  0 17 14]
 [ 0 15 16 17  4 18]]


In [24]:
# '১' মানে পজিটিভ, '০' মানে নেগেটিভ, কারণ pd.get_dummy() দেখিয়েছে কলাম '০' মানে নেগেটিভ আর কলাম '১' মানে পজিটিভ  

np.argmax(prediction, axis=1)

array([5, 5, 5, 5, 4, 5], dtype=int64)